In [1]:
import pandas as pd

# The data as a list of lists
data = [
    ['male', 'C', 12.40],
    ['female', 'A', 7.69],
    ['male', 'C', 14.01],
    ['female', 'A', 9.69],
    ['male', 'C', 11.65],
    ['female', 'A', 8.89],
    ['female', 'A', 6.94],
    ['female', 'A', 2.13],
    ['female', 'A', 7.26],
    ['female', 'A', 5.87],
    ['male', 'B', 12.93],
    ['female', 'C', 12.19],
    ['female', 'A', 7.20],
    ['male', 'C', 13.88],
    ['male', 'A', 8.18],
    ['male', 'B', 16.64],
    ['female', 'C', 9.41],
    ['male', 'C', 11.21],
    ['female', 'B', 8.35],
    ['male', 'A', 7.24],
    ['female', 'A', 6.81],
    ['male', 'B', 9.81],
    ['female', 'A', 6.67],
    ['female', 'A', 6.98],
    ['female', 'A', 7.07],
    ['female', 'C', 2.40],
    ['male', 'B', 7.84],
    ['female', 'B', 3.84],
    ['male', 'B', 9.42],
    ['male', 'A', 7.00],
    ['male', 'A', 7.00],
    ['female', 'A', 5.00],
    ['male', 'A', 8.00]
]

# Create the DataFrame
df = pd.DataFrame(data, columns=['Gender', 'Group', 'Value'])

# Convert 'Value' column to numeric type
df['Value'] = pd.to_numeric(df['Value'])

# Display the first few rows of the DataFrame
print(df.head())

# Display basic information about the DataFrame
print(df.info())

# Display summary statistics of the DataFrame
print(df.describe())


   Gender Group  Value
0    male     C  12.40
1  female     A   7.69
2    male     C  14.01
3  female     A   9.69
4    male     C  11.65
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Gender  33 non-null     object 
 1   Group   33 non-null     object 
 2   Value   33 non-null     float64
dtypes: float64(1), object(2)
memory usage: 924.0+ bytes
None
           Value
count  33.000000
mean    8.533333
std     3.253118
min     2.130000
25%     6.980000
50%     7.840000
75%     9.810000
max    16.640000


In [2]:
from utilities import group_by2factors
import statistics as stat

d=group_by2factors(df["Gender"],df["Group"],df["Value"])

In [41]:
from scipy.stats import f as f_stat
from prettytable import PrettyTable as PT
import statistics as stat
######## Own Modules ##########
from utilities import group_by2factors

def two_way_anova(data,factor_listA=None,
factor_listB=None, print_out=False,name_a="A",name_b="B"):
    """
    The data input should be a 3D list, the first two dims are the
    two factors. If the factor_listA and B are inputted, the data should be a 
    1D list.
    """
    if factor_listA is not None and factor_listB is not None:
        d=group_by2factors(factor_listA,factor_listB,data)
    else:
        d=data
    flat_list=[k for i in d for j in i for k in j]
    n=len(flat_list)
    a=len(d)
    b=len(d[0])
    y_bar_a=[stat.mean([k for j in i for k in j]) for i in d]
    y_bar_b=[stat.mean([k for j in range(a) for k in d[j][i]])
            for i in range(b)]
    n_a=[len([k for j in i for k in j]) for i in d]
    n_b=[len([k for j in range(a) for k in d[j][i]])
            for i in range(b)]
    y_bar=stat.mean(flat_list)
    SSE=sum([sum((x-stat.mean(k))**2 for x in k) for j in d for k in j])
    SST=sum((x-y_bar)**2 for j in d for k in j for x in k)
#     SSA=n*b*sum((i-y_bar)**2 for i in y_bar_a)
    SSA=sum(j*(i-y_bar)**2 for i,j in zip(y_bar_a,n_a))
#     SSB=n*a*sum((i-y_bar)**2 for i in y_bar_b)
    SSB=sum(j*(i-y_bar)**2 for i,j in zip(y_bar_b,n_b))
    Re_SS_B=sum([(k-y_bar_b[bi])**2 for ai in range(a)
                for bi in range(b) for k in d[ai][bi]])
    print(SST-Re_SS_B)
    
    SSAB=SST-SSE-SSA-SSB
    DFA=a-1
    DFB=b-1
    DFAB=DFA*DFB
    DFT=n-1
    DFE=DFT-DFA-DFB-DFAB
    MSA=SSA/DFA
    MSB=SSB/DFB
    MSAB=SSAB/DFAB
    MSE=SSE/DFE
    FA=MSA/MSE
    FB=MSB/MSE
    FAB=MSAB/MSE
    PA=1-f_stat.cdf(FA,DFA,DFE)
    PB=1-f_stat.cdf(FB,DFB,DFE)
    PAB=1-f_stat.cdf(FAB,DFAB,DFE)
    print(y_bar,y_bar_a,y_bar_b,n_b,n_a)
    
    if print_out:
        print("Two Way ANOVA\n\nFactor Info.")
        t=PT()
        t.field_names=["Factor","Levels"]
        t.add_row([name_a, a])
        t.add_row([name_b, b])
        print(t)
        print("\nAnalysis of Variance")
        t=PT()
        t.field_names=["Source","DF","Adj SS","Adj MS",
        "F-value","p-value"]
        t.add_row([name_a,DFA,"%.3f"%SSA,"%.3f"%MSA,"%.3f"%FA,"%.3f"%PA])
        t.add_row([name_b,DFB,"%.3f"%SSB,"%.3f"%MSB,"%.3f"%FB,"%.3f"%PB])
        t.add_row([name_a+"*"+name_b,DFAB,"%.3f"%SSAB,"%.3f"%MSAB,"%.3f"%FAB,"%.3f"%PAB])
        t.add_row(["Error", DFE, "%.3f"%SSE,"%.3f"%MSE," "," "])
        t.add_row(["Total",DFT,"%.3f"%SST," "," "," "])
        print(t)\
    
    return {"N":n,"SSE":SSE,"MSE":MSE,"SSA":SSA,"MSA":MSA,
    "SSB":SSB,"SSAB":SSAB,"MSB":MSB,"MSAB":MSAB,"DF A":DFA,
    "DF B":DFB, "DF AB":DFAB,"F A":FA,"F B":FB,"F AB":FAB,
    "p A":PA,"p B":PB,"p AB":PAB}
    


In [42]:
_=two_way_anova(d,print_out=True)

99.8872251984127
8.533333333333333 [6.910555555555556, 10.480666666666666] [6.978888888888889, 9.832857142857144, 10.89375] [18, 7, 8] [18, 15]
Two Way ANOVA

Factor Info.
+--------+--------+
| Factor | Levels |
+--------+--------+
|   A    |   2    |
|   B    |   3    |
+--------+--------+

Analysis of Variance
+--------+----+---------+---------+---------+---------+
| Source | DF |  Adj SS |  Adj MS | F-value | p-value |
+--------+----+---------+---------+---------+---------+
|   A    | 1  | 104.283 | 104.283 |  17.857 |  0.000  |
|   B    | 2  |  99.887 |  49.944 |  8.552  |  0.001  |
|  A*B   | 2  | -23.202 | -11.601 |  -1.986 |  1.000  |
| Error  | 27 | 157.681 |  5.840  |         |         |
| Total  | 32 | 338.649 |         |         |         |
+--------+----+---------+---------+---------+---------+


In [37]:
data = [
    # Variety 1
    [
        [7.8, 9.1, 10.6],   # 5 k/ha
        [11.2, 12.7, 13.3], # 10 k/ha
        [12.1, 12.5, 14.1], # 15 k/ha
        [9.1, 10.7, 12.6]   # 20 k/ha
    ],
    # Variety 2
    [
        [8.0, 8.7, 10.0],   # 5 k/ha
        [11.3, 12.9, 13.8], # 10 k/ha
        [13.8, 14.3, 15.4], # 15 k/ha
        [11.3, 12.7, 14.3]  # 20 k/ha
    ],
    # Variety 3
    [
        [15.3, 16.0, 17.6], # 5 k/ha
        [16.8, 18.3, 19.2], # 10 k/ha
        [17.9, 21.0, 20.7], # 15 k/ha
        [17.2, 18.3, 19.1]  # 20 k/ha
    ]
]
two_way_anova(data,print_out=True)

13.880555555555556 [11.316666666666666, 12.208333333333334, 18.116666666666667] [11.455555555555556, 14.38888888888889, 15.755555555555555, 13.922222222222222]
Two Way ANOVA

Factor Info.
+--------+--------+
| Factor | Levels |
+--------+--------+
|   A    |   3    |
|   B    |   4    |
+--------+--------+

Analysis of Variance
+--------+----+---------+---------+---------+---------+
| Source | DF |  Adj SS |  Adj MS | F-value | p-value |
+--------+----+---------+---------+---------+---------+
|   A    | 2  | 327.774 | 163.887 | 100.476 |  0.000  |
|   B    | 3  |  86.907 |  28.969 |  17.760 |  0.000  |
|  A*B   | 6  |  8.068  |  1.345  |  0.824  |  0.562  |
| Error  | 24 |  39.147 |  1.631  |         |         |
| Total  | 35 | 461.896 |         |         |         |
+--------+----+---------+---------+---------+---------+


{'N': 36,
 'SSE': 39.146666666666675,
 'MSE': 1.6311111111111114,
 'SSA': 327.7738888888889,
 'MSA': 163.88694444444445,
 'SSB': 86.90749999999997,
 'SSAB': 8.068333333333328,
 'MSB': 28.969166666666656,
 'MSAB': 1.3447222222222213,
 'DF A': 2,
 'DF B': 3,
 'DF AB': 6,
 'F A': 100.47564713896456,
 'F B': 17.760388283378738,
 'F AB': 0.8244209809264298,
 'p A': 2.175037927543144e-12,
 'p B': 2.7384098582050598e-06,
 'p AB': 0.5623027998578893}

In [48]:
import pandas as pd
 

# Assuming we have a DataFrame 'df' with columns 'Y' (dependent variable), 'A' (Factor A), and 'B' (Factor B)
data = [
    ['male', 'C', 12.40],
    ['female', 'A', 7.69],
    ['male', 'C', 14.01],
    ['female', 'A', 9.69],
    ['male', 'C', 11.65],
    ['female', 'A', 8.89],
    ['female', 'A', 6.94],
    ['female', 'A', 2.13],
    ['female', 'A', 7.26],
    ['female', 'A', 5.87],
    ['male', 'B', 12.93],
    ['female', 'C', 12.19],
    ['female', 'A', 7.20],
    ['male', 'C', 13.88],
    ['male', 'A', 8.18],
    ['male', 'B', 16.64],
    ['female', 'C', 9.41],
    ['male', 'C', 11.21],
    ['female', 'B', 8.35],
    ['male', 'A', 7.24],
    ['female', 'A', 6.81],
    ['male', 'B', 9.81],
    ['female', 'A', 6.67],
    ['female', 'A', 6.98],
    ['female', 'A', 7.07],
    ['female', 'C', 2.40],
    ['male', 'B', 7.84],
    ['female', 'B', 3.84],
    ['male', 'B', 9.42],
    ['male', 'A', 7.00],
    ['male', 'A', 7.00],
    ['female', 'A', 5.00],
    ['male', 'A', 8.00]
]

# Create the DataFrame
df = pd.DataFrame(data, columns=['A', 'B', 'Y'])

# Step 1: Fit the full model
full_model = ols("Y ~ C(A, Sum) + C(B, Sum) + C(A, Sum):C(B, Sum)",
                 data=df).fit()

# Step 2: Fit the reduced model (without Factor A)
reduced_model = ols("Y ~ C(B, Sum) + C(A, Sum):C(B, Sum)",
                    data=df).fit()

# Step 3: Calculate Type 3 SS for Factor A
ss_full = ((full_model.fittedvalues - df['Y'].mean())**2).sum()
ss_reduced = ((reduced_model.fittedvalues - df['Y'].mean())**2).sum()
type3_ss_A = ss_full - ss_reduced

# Print the result
print(f"Type 3 Sum of Squares for Factor A: {type3_ss_A}")

# To get the complete ANOVA table with Type 3 SS
anova_table = sm.stats.anova_lm(full_model, typ=3)
print(anova_table)
print(full_model)
dir(anova_table)

Type 3 Sum of Squares for Factor A: -5.684341886080802e-14
                          sum_sq    df           F        PR(>F)
Intercept            1812.640170   1.0  310.382069  2.470703e-16
C(A, Sum)              73.870879   1.0   12.649061  1.412004e-03
C(B, Sum)              51.032793   2.0    4.369225  2.270277e-02
C(A, Sum):C(B, Sum)    30.603340   2.0    2.620136  9.120704e-02
Residual              157.680773  27.0         NaN           NaN


['F',
 'T',
 '_AXIS_LEN',
 '_AXIS_ORDERS',
 '_AXIS_TO_AXIS_NUMBER',
 '_HANDLED_TYPES',
 '__abs__',
 '__add__',
 '__and__',
 '__annotations__',
 '__array__',
 '__array_priority__',
 '__array_ufunc__',
 '__arrow_c_stream__',
 '__bool__',
 '__class__',
 '__contains__',
 '__copy__',
 '__dataframe__',
 '__dataframe_consortium_standard__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__finalize__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__imod__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pandas_priority__',
 '__pos__',
 '__pow__',

In [55]:
anova_table.sum_sq.iloc[2]

51.03279330758174

In [56]:
anova_table.sum_sq

Intercept              1812.640170
C(A, Sum)                73.870879
C(B, Sum)                51.032793
C(A, Sum):C(B, Sum)      30.603340
Residual                157.680773
Name: sum_sq, dtype: float64

In [3]:
from two_way_anova import two_way_anova as t2
_=t2(d,print_out=True)

Two Way ANOVA

Factor Info.
+--------+--------+
| Factor | Levels |
+--------+--------+
|   A    |   2    |
|   B    |   3    |
+--------+--------+

Analysis of Variance
+--------+----+---------+--------+---------+---------+
| Source | DF |  Adj SS | Adj MS | F-value | p-value |
+--------+----+---------+--------+---------+---------+
|   A    | 1  |  73.871 | 73.871 |  12.649 |  0.001  |
|   B    | 2  |  51.033 | 25.516 |  4.369  |  0.023  |
|  A*B   | 2  |  30.603 | 15.302 |  2.620  |  0.091  |
| Error  | 27 | 157.681 | 5.840  |         |         |
| Total  | 32 | 338.649 |        |         |         |
+--------+----+---------+--------+---------+---------+


In [5]:
data = [line.split('\t') for line in '''Male	Married	34
Male	Single	36
Male	Married	23
Male	Single	29
Male	Married	39
Male	Single	34
Female	Married	42
Female	Married	40
Male	Married	28
Female	Married	26
Female	Married	26
Male	Single	26
Female	Married	37
Male	Single	25
Female	Single	27
Male	Married	41
Male	Single	35
Male	Single	23
Male	Married	28
Male	Married	38
Female	Married	25
Male	Married	30
Male	Married	36
Female	Single	29
Female	Married	31
Female	Married	26
Male	Single	23
Female	Married	31
Female	Married	31
Male	Married	35
Female	Married	25
Male	Single	21
Female	Married	28
Male	Married	27
Male	Married	35
Male	Single	24
Female	Single	29
Male	Married	34
Female	Married	28
Male	Married	37
Female	Married	25
Female	Married	24
Female	Single	23
Female	Single	25
Male	Married	34
Male	Single	25
Female	Married	31
Male	Single	26
Male	Married	31
Male	Married	34
Female	Married	29
Male	Married	33
Female	Married	32
Male	Married	36
Male	Single	32
Male	Married	27
Male	Married	28
Male	Single	27
Male	Married	36
Female	Married	27
Male	Married	37
Female	Single	31
Male	Married	29
Male	Married	38
Male	Married	37
Male	Married	38
Male	Married	31
Male	Married	39
Female	Married	37
Male	Single	36
Female	Married	37
Male	Single	27
Male	Single	26
Female	Married	23
Female	Single	28
Female	Single	29
Male	Married	35
Female	Married	40
Male	Married	44
Male	Married	35
Male	Married	30
Male	Married	27
Female	Single	24
Female	Married	35
Female	Married	26
Female	Married	32
Male	Single	22
Female	Married	25
Male	Married	33
Female	Married	37
Male	Married	27
Male	Married	38
Female	Single	25
Male	Single	24
Male	Married	25
Male	Married	28
Male	Married	20
Female	Married	25
Female	Married	39
Male	Married	28
Female	Married	32
Male	Married	22
Male	Married	28
Female	Married	27
Male	Married	38
Male	Married	33
Male	Single	35
Female	Married	24
Male	Married	27
Female	Single	40
Female	Married	27
Female	Single	25
Male	Single	26
Male	Single	30
Male	Single	26
Male	Single	23
Male	Single	24
Male	Married	28
Male	Married	43
Male	Single	36
Female	Single	24
Female	Single	60
Female	Single	24
Female	Married	30
Male	Married	30
Male	Married	31
Female	Married	28
Male	Single	26
Female	Single	41
Female	Single	35
Female	Single	25
Male	Single	31
Female	Married	30
Female	Married	34
Female	Married	33
Male	Single	31
Male	Married	33
Male	Married	33
Female	Married	28
Female	Married	32
Female	Married	39
Female	Married	29
Female	Single	21
Female	Married	29
Female	Married	26
Female	Single	23
Female	Married	29
Male	Single	35
Male	Married	33
Female	Single	33
Male	Married	29
Male	Single	23
Male	Single	35
Female	Married	24
Male	Married	39
Male	Married	33
Male	Single	30
Female	Married	26
Female	Married	36
Female	Married	25
Male	Single	23
Female	Married	37
Male	Married	34
Female	Married	31
Female	Married	39
Female	Married	41
Female	Married	39
Male	Married	34
Female	Single	32
Female	Married	29
Female	Single	23
Male	Married	41
Female	Married	27
Male	Single	26
Male	Married	28
Female	Married	37
Female	Single	39
Male	Married	32
Male	Married	34
Female	Single	29
Male	Single	25
Female	Married	30
Female	Married	29
Male	Married	32
Female	Married	31
Male	Single	22
Male	Married	31
Male	Married	31
Female	Married	40
Female	Married	26
Male	Single	23
Male	Married	35
Male	Single	26
Female	Married	29
Male	Single	26
Female	Single	31
Male	Married	25
Female	Single	45
Female	Single	33
Male	Single	38
Male	Single	25
Male	Married	32
Male	Single	24
Female	Married	33
Male	Married	33
Female	Married	42
Female	Married	27
Male	Married	33
Female	Single	33
Male	Married	32
Male	Single	28
Female	Single	30
Female	Single	26
Male	Single	27
Male	Married	23
Female	Married	27
Male	Married	33
Male	Married	24
Male	Single	22
Female	Single	28
Male	Married	32
Male	Single	24
Male	Married	29
Male	Married	32
Male	Married	30
Female	Married	35
Female	Single	32
Male	Married	32
Male	Married	40
Male	Single	30
Male	Single	29
Female	Married	32
Female	Married	35
Female	Single	25
Male	Single	25
Female	Single	23
Female	Single	22
Male	Single	26
Male	Married	25
Male	Single	30
Female	Married	29
Female	Married	32
Female	Single	18
Female	Married	25
Male	Married	21
Female	Married	37
Male	Married	38
Male	Single	32
Male	Single	34
Male	Married	35
Male	Married	34
Male	Single	46
Male	Married	37
Male	Single	37
Male	Married	35
Male	Single	54
Male	Single	36
Female	Married	32
Female	Married	37
Male	Married	27
Female	Married	25
Male	Single	28
Male	Single	35
Female	Single	27
Female	Single	24
Female	Married	37
Male	Married	30
Female	Single	28
Female	Married	27
Female	Married	26
Female	Married	51
Male	Single	28
Male	Married	34
Male	Married	32
Female	Married	27
Female	Single	33
Female	Married	26
Male	Married	23
Male	Married	28
Male	Single	30
Female	Married	21
Female	Married	32
Female	Married	30
Male	Single	46
Female	Married	25
Female	Married	25
Female	Married	40
Female	Single	46
Female	Single	34
Female	Married	37
Male	Married	33
Female	Married	27
Female	Married	25
Female	Married	35
Male	Married	34
Male	Single	29
Male	Married	29
Male	Married	26
Male	Married	25
Male	Married	31
Female	Married	25
Male	Married	32
Male	Married	29'''.split('\n')]

# Now, create the DataFrame
df = pd.DataFrame(data, columns=['Gender', 'Marital_Status', 'Age'])
d=group_by2factors(df["Gender"],df["Marital_Status"],
                   [float(i) for i in df["Age"]])
t2(d,print_out=True)

Two Way ANOVA

Factor Info.
+--------+--------+
| Factor | Levels |
+--------+--------+
|   A    |   2    |
|   B    |   2    |
+--------+--------+

Analysis of Variance
+--------+-----+-----------+---------+---------+---------+
| Source |  DF |   Adj SS  |  Adj MS | F-value | p-value |
+--------+-----+-----------+---------+---------+---------+
|   A    |  1  |   1.919   |  1.919  |  0.055  |  0.815  |
|   B    |  1  |  250.765  | 250.765 |  7.157  |  0.008  |
|  A*B   |  1  |   46.061  |  46.061 |  1.315  |  0.252  |
| Error  | 299 | 10476.473 |  35.038 |         |         |
| Total  | 302 | 10815.155 |         |         |         |
+--------+-----+-----------+---------+---------+---------+


{'N': 303,
 'SSE': 10476.47346349843,
 'MSE': 35.03837278762017,
 'SSA': 1.919425138443521,
 'MSA': 1.919425138443521,
 'SSB': 250.7652487578956,
 'SSAB': 46.06055154699126,
 'MSB': 250.7652487578956,
 'MSAB': 46.06055154699126,
 'DF A': 1,
 'DF B': 1,
 'DF AB': 1,
 'F A': 0.05478065862469779,
 'F B': 7.156874843414432,
 'F AB': 1.314574504534796,
 'p A': 0.8151042555672676,
 'p B': 0.007879011872439334,
 'p AB': 0.2524847847063927}

In [11]:
d1=[[ij[:10] for ij in j] for j in d]
_=t2(d1,print_out=True)

Two Way ANOVA

Factor Info.
+--------+--------+
| Factor | Levels |
+--------+--------+
|   A    |   2    |
|   B    |   2    |
+--------+--------+

Analysis of Variance
+--------+----+----------+---------+---------+---------+
| Source | DF |  Adj SS  |  Adj MS | F-value | p-value |
+--------+----+----------+---------+---------+---------+
|   A    | 1  |  13.225  |  13.225 |  0.485  |  0.491  |
|   B    | 1  | 255.025  | 255.025 |  9.344  |  0.004  |
|  A*B   | 1  |  3.025   |  3.025  |  0.111  |  0.741  |
| Error  | 36 | 982.500  |  27.292 |         |         |
| Total  | 39 | 1253.775 |         |         |         |
+--------+----+----------+---------+---------+---------+


In [7]:
d1

[[[42.0, 40.0, 26.0, 26.0, 37.0, 25.0, 31.0, 26.0, 31.0, 31.0],
  [27.0, 29.0, 29.0, 23.0, 25.0, 31.0, 28.0, 29.0, 24.0, 25.0]],
 [[34.0, 23.0, 39.0, 28.0, 41.0, 28.0, 38.0, 30.0, 36.0, 35.0],
  [36.0, 29.0, 34.0, 26.0, 25.0, 35.0, 23.0, 23.0, 21.0, 24.0]]]

In [10]:
l=[]
for i, ith_A in enumerate(d1):
    for j, ith_A_jth_B in enumerate(ith_A):
        for k in ith_A_jth_B:
            l.append([i, j, k])
df = pd.DataFrame(l, columns=["A", "B", "Y"])
print(df)

    A  B     Y
0   0  0  42.0
1   0  0  40.0
2   0  0  26.0
3   0  0  26.0
4   0  0  37.0
5   0  0  25.0
6   0  0  31.0
7   0  0  26.0
8   0  0  31.0
9   0  0  31.0
10  0  1  27.0
11  0  1  29.0
12  0  1  29.0
13  0  1  23.0
14  0  1  25.0
15  0  1  31.0
16  0  1  28.0
17  0  1  29.0
18  0  1  24.0
19  0  1  25.0
20  1  0  34.0
21  1  0  23.0
22  1  0  39.0
23  1  0  28.0
24  1  0  41.0
25  1  0  28.0
26  1  0  38.0
27  1  0  30.0
28  1  0  36.0
29  1  0  35.0
30  1  1  36.0
31  1  1  29.0
32  1  1  34.0
33  1  1  26.0
34  1  1  25.0
35  1  1  35.0
36  1  1  23.0
37  1  1  23.0
38  1  1  21.0
39  1  1  24.0
